In [ ]:
# Procudes a pickle that has zero counts continuous rows of zero meter reading.
# If there is 85 meter reading in a row with 0 all 85 readings will be marked 85
# in the count colunm.
# df[['building_id','meter', 'timestamp','start_zero','end_zero','count_zero']]

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from datetime import date 
import holidays


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [2]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [21]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [22]:
building = loadFile('building_metadata')
pre_train = loadFile('train')
df = pre_train.sort_values(by=['building_id','meter','timestamp'])
#test = loadFile('test')

In [23]:
df = df.join(df.shift(-1), rsuffix='_m_1') # minus 1
df = df.join(df.shift(1), rsuffix='_p_1') # plus 1

print(df.head)

<bound method NDFrame.head of           building_id  meter           timestamp  meter_reading  \
0                   0      0 2016-01-01 00:00:00          0.000   
2301                0      0 2016-01-01 01:00:00          0.000   
4594                0      0 2016-01-01 02:00:00          0.000   
6893                0      0 2016-01-01 03:00:00          0.000   
9189                0      0 2016-01-01 04:00:00          0.000   
11485               0      0 2016-01-01 05:00:00          0.000   
13780               0      0 2016-01-01 06:00:00          0.000   
16073               0      0 2016-01-01 07:00:00          0.000   
18367               0      0 2016-01-01 08:00:00          0.000   
20661               0      0 2016-01-01 09:00:00          0.000   
22956               0      0 2016-01-01 10:00:00          0.000   
25253               0      0 2016-01-01 11:00:00          0.000   
27550               0      0 2016-01-01 12:00:00          0.000   
29847               0      0 201

[20216100 rows x 16 columns]>


In [24]:
%%time
#df = df.head(2000)

df['zero_start'] = np.nan
df['zero_end'] = np.nan
df['continuous_0s'] = 0

Wall time: 226 ms


In [25]:
print(df[['zero_start','zero_end']])

          zero_start  zero_end
0                NaN       NaN
2301             NaN       NaN
4594             NaN       NaN
6893             NaN       NaN
9189             NaN       NaN
11485            NaN       NaN
13780            NaN       NaN
16073            NaN       NaN
18367            NaN       NaN
20661            NaN       NaN
22956            NaN       NaN
25253            NaN       NaN
27550            NaN       NaN
29847            NaN       NaN
32145            NaN       NaN
34445            NaN       NaN
36741            NaN       NaN
39041            NaN       NaN
41340            NaN       NaN
43637            NaN       NaN
45933            NaN       NaN
48230            NaN       NaN
50526            NaN       NaN
52823            NaN       NaN
55121            NaN       NaN
57420            NaN       NaN
59715            NaN       NaN
62012            NaN       NaN
64310            NaN       NaN
66606            NaN       NaN
...              ...       ...
20147574

In [26]:
zeros_df = df[df['meter_reading'] == 0.0]
print(df.head())

      building_id  meter           timestamp  meter_reading  building_id_m_1  \
0               0      0 2016-01-01 00:00:00            0.0              0.0   
2301            0      0 2016-01-01 01:00:00            0.0              0.0   
4594            0      0 2016-01-01 02:00:00            0.0              0.0   
6893            0      0 2016-01-01 03:00:00            0.0              0.0   
9189            0      0 2016-01-01 04:00:00            0.0              0.0   

      meter_m_1       timestamp_m_1  meter_reading_m_1  building_id_p_1  \
0           0.0 2016-01-01 01:00:00                0.0              NaN   
2301        0.0 2016-01-01 02:00:00                0.0              0.0   
4594        0.0 2016-01-01 03:00:00                0.0              0.0   
6893        0.0 2016-01-01 04:00:00                0.0              0.0   
9189        0.0 2016-01-01 05:00:00                0.0              0.0   

      meter_p_1       timestamp_p_1  meter_reading_p_1  building_id_

In [27]:
start_zero = None
start_zeros = []
for row in zeros_df.itertuples():
    if start_zero is None:
        start_zero = row.timestamp
    elif row.building_id != row.building_id_p_1:
        print(f"new building {row.building_id}")
        start_zero = row.timestamp
    elif row.meter != row.meter_p_1:
        start_zero = row.timestamp
    elif row.meter_reading_p_1 != 0.0:
        start_zero = row.timestamp        
    start_zeros.append(start_zero)



new building 1
new building 2
new building 3
new building 4
new building 5
new building 6
new building 7
new building 8
new building 9
new building 10
new building 11
new building 12
new building 13
new building 14
new building 15
new building 16
new building 17
new building 18
new building 19
new building 20
new building 21
new building 22
new building 23
new building 24
new building 25
new building 26
new building 27
new building 28
new building 29
new building 30
new building 31
new building 32
new building 33
new building 34
new building 35
new building 36
new building 37
new building 38
new building 39
new building 40
new building 41
new building 42
new building 43
new building 44
new building 45
new building 47
new building 48
new building 49
new building 50
new building 51
new building 52
new building 53
new building 54
new building 55
new building 56
new building 57
new building 58
new building 59
new building 60
new building 61
new building 62
new building 63
new building 64
n

In [28]:
print(start_zeros[0:1000])
zeros_df['start_zero']=start_zeros

[Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('2016-01-01 00:00:00'), Timestamp('20

In [29]:
end_zero = None
end_zeros = []
for row in reversed(list(zeros_df.itertuples())):
    if end_zero is None:
        end_zero = row.timestamp
    elif row.building_id != row.building_id_m_1:
        print(f"new building {row.building_id}")
        end_zero = row.timestamp
    elif row.meter != row.meter_m_1:
        end_zero = row.timestamp
    elif row.meter_reading_m_1 != 0.0:
        end_zero = row.timestamp        
    end_zeros.append(end_zero)

end_zeros = list(reversed(end_zeros))

new building 1411
new building 1398
new building 1397
new building 1395
new building 1346
new building 1307
new building 1302
new building 1280
new building 1264
new building 1262
new building 1260
new building 1258
new building 1255
new building 1253
new building 1234
new building 1233
new building 1217
new building 1216
new building 1208
new building 1207
new building 1201
new building 1196
new building 1177
new building 1161
new building 1146
new building 1144
new building 1142
new building 1137
new building 1132
new building 1127
new building 1122
new building 1119
new building 1116
new building 1111
new building 1101
new building 1095
new building 1084
new building 1083
new building 1074
new building 1017
new building 1013
new building 1003
new building 1001
new building 994
new building 993
new building 991
new building 980
new building 975
new building 963
new building 933
new building 926
new building 918
new building 900
new building 857
new building 853
new building 780
new b

In [30]:
print(end_zeros[0:1000])
zeros_df['end_zero']=end_zeros

[Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('2016-01-30 07:00:00'), Timestamp('20

In [31]:
df['start_zero'] = zeros_df['start_zero']
df['end_zero'] = zeros_df['end_zero']

In [32]:
df[['start_zero','end_zero']].head(1000)

,start_zero,end_zero
0,2016-01-01 00:00:00,2016-01-30 07:00:00
2301,2016-01-01 00:00:00,2016-01-30 07:00:00
4594,2016-01-01 00:00:00,2016-01-30 07:00:00
6893,2016-01-01 00:00:00,2016-01-30 07:00:00
9189,2016-01-01 00:00:00,2016-01-30 07:00:00
11485,2016-01-01 00:00:00,2016-01-30 07:00:00
13780,2016-01-01 00:00:00,2016-01-30 07:00:00
16073,2016-01-01 00:00:00,2016-01-30 07:00:00
18367,2016-01-01 00:00:00,2016-01-30 07:00:00
20661,2016-01-01 00:00:00,2016-01-30 07:00:00


In [33]:
df.loc[(df['end_zero'] < df['start_zero']),['start_zero','end_zero']]

,start_zero,end_zero


In [34]:
df['count_zero'] = (df['end_zero'] - df['start_zero'])/ np.timedelta64(1, 'h')
print(df[['count_zero']].sample(200))

          count_zero
19210096         NaN
11669850         NaN
17425960         NaN
17867757         NaN
8778780          NaN
7375454          NaN
11560210         NaN
17216023         NaN
11845558         NaN
16604167         NaN
758778           NaN
19244672         NaN
15807608         NaN
3744656          NaN
7338801          NaN
2114345       2922.0
6968523          NaN
3288834          NaN
8824867          NaN
7158758          NaN
19549326         NaN
7586410          NaN
9120344          NaN
9219528          NaN
3614172          NaN
13786065         NaN
16727958         NaN
8073915          NaN
17130056         NaN
1790183          NaN
...              ...
3846122          NaN
14628002         NaN
8296376          NaN
8158664          NaN
6469059          NaN
9143147         11.0
13355438         NaN
14613921         NaN
5747851          NaN
12072444         NaN
12445249         NaN
3006389       1153.0
8870912          3.0
6685106          NaN
16934919         NaN
10455808     

In [37]:
out_df = df[['building_id','meter', 'timestamp','start_zero','end_zero','count_zero']].sort_index()
print(out_df)
out_df.to_pickle('../input/ashrae-energy-prediction-pickles/zero_counts.pickle')

          building_id  meter           timestamp          start_zero  \
0                   0      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
1                   1      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
2                   2      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
3                   3      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
4                   4      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
5                   5      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
6                   6      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
7                   7      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
8                   8      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
9                   9      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
10                 10      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
11                 11      0 2016-01-01 00:00:00 2016-01-01 00:00:00   
12                 12      0 2016-01-01 00:00:00 2016-01-01 00:0